# MobileNetV2 Pretraining on UnityEyes

In [ ]:
!pip uninstall -y tensorflow
!pip install tensorflow==2.18.0
!pip install tensorflowjs

Found existing installation: tensorflow 2.18.0
Uninstalling tensorflow-2.18.0:
  Successfully uninstalled tensorflow-2.18.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 615.4/615.4 MB 1.8 MB/s eta 0:00:00


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.1/89.1 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 5.1 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 24.2
    Uninstalling packaging-24.2:
      Successfully uninstalled packaging-24.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
db-dtypes 1.4.3 requires packaging>=24.2.0, but you have packaging 23.2 which is incompatible.
google-cloud-bigquery 3.32.0 requires packaging>=24.2.0, but you have packaging 23.2 which is incompatible.


In [ ]:
import tensorflow as tf
import tensorflowjs as tfjs
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
import os, shutil, random
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.utils.class_weight import compute_class_weight

random.seed(42028)

In [ ]:
!nvidia-smi

Sun May 25 09:55:01 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off |   00000000:00:04.0 Off |                    0 |
| N/A   34C    P0             47W /  400W |       0MiB /  40960MiB |      0%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+-----

In [ ]:
print("TF Version:", tf.__version__)
print("GPU:", tf.config.list_physical_devices('GPU'))

TF Version: 2.18.0
GPU: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
PROJECT_DIR = "/content/drive/Shareddrives/OmniClick Team"
DATA_ROOT_DIR = os.path.join(PROJECT_DIR, "datasets")
UNITY_DATA_DIR = os.path.join(DATA_ROOT_DIR, 'UnityEyes')
UNITY_SUBSET_DIR = os.path.join(DATA_ROOT_DIR, 'UnityEyesSubset')

BEST_MODEL_PATH = os.path.join(PROJECT_DIR, "notebooks/weights/mobilenetv2_unity_best.keras")
LAST_MODEL_PATH = os.path.join(PROJECT_DIR, "notebooks/weights/mobilenetv2_unity_last.keras")
TFJS_MODEL_DIR = os.path.join(PROJECT_DIR, "notebooks/weights/mobilenetv2_unity_tfjs")
TF_SAVE_MODEL_DIR = os.path.join(PROJECT_DIR, "notebooks/weights/mobilenetv2_unity_tf")

In [ ]:
%ls "$DATA_ROOT_DIR"

columbia_gaze_data_set.zip  OurWebcamDataset/  UnityEyesSubset/  u_train.zip
ColumbiaGazeProcessed/      UnityEyes/         u_test.zip        u_val.zip


In [ ]:
def subsample_directory(src_dir, dst_dir, sample_fraction=0.2):
    os.makedirs(dst_dir, exist_ok=True)

    for class_name in sorted(os.listdir(src_dir)):
        src_class = os.path.join(src_dir, class_name)
        dst_class = os.path.join(dst_dir, class_name)
        os.makedirs(dst_class, exist_ok=True)

        all_images = os.listdir(src_class)
        sampled_images = random.sample(all_images, int(len(all_images) * sample_fraction))

        for img in sampled_images:
            shutil.copy(os.path.join(src_class, img), os.path.join(dst_class, img))

# for split in ['train', 'val', 'test']:
for split in ['test']:
    subsample_directory(
        src_dir=os.path.join(UNITY_DATA_DIR, split),
        dst_dir=os.path.join(UNITY_SUBSET_DIR, split),
        sample_fraction=0.1  # 10% subset
    )

In [ ]:
IMG_SIZE = (96, 96)
BATCH_SIZE = 768

unity_train_dir = os.path.join(UNITY_SUBSET_DIR, 'train')
unity_val_dir = os.path.join(UNITY_SUBSET_DIR, 'val')
unity_test_dir = os.path.join(UNITY_SUBSET_DIR, 'test')
unity_class_names = sorted(os.listdir(unity_train_dir))
unity_class_indices = {name: idx for idx, name in enumerate(unity_class_names)}
NUM_CLASSES_UNITY = len(unity_class_names)

# Data Aug for Unity set
unity_train_gen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=10,
    brightness_range=[0.7, 1.3],
    zoom_range=0.1,
    horizontal_flip=True
).flow_from_directory(
    unity_train_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)
unity_val_gen = ImageDataGenerator(rescale=1./255).flow_from_directory(
    unity_val_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)
unity_test_gen = ImageDataGenerator(rescale=1./255).flow_from_directory(
    unity_test_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False
)

# Checkpoints
callbacks = [
    ModelCheckpoint(
        BEST_MODEL_PATH,
        monitor='val_loss',
        save_best_only=True,
        save_weights_only=False,
        verbose=1
    ),
    ModelCheckpoint(
        LAST_MODEL_PATH,
        save_best_only=False,
        save_weights_only=False,
        verbose=1
    ),
    EarlyStopping(
        monitor='val_loss',
        patience=20,
        restore_best_weights=True
    )
]

Found 6103 images belonging to 8 classes.
Found 7247 images belonging to 8 classes.
Found 8232 images belonging to 5 classes.


In [ ]:
base_model = MobileNetV2(input_shape=IMG_SIZE + (3,), include_top=False, weights='imagenet')
base_model.trainable = True

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.3)(x)
output = Dense(NUM_CLASSES_UNITY, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=output)
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step


In [ ]:
# Train
history = model.fit(
    unity_train_gen,
    validation_data=unity_val_gen,
    epochs=100,
    callbacks=callbacks
)
model.export(TF_SAVE_MODEL_DIR)

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/100
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 432s/step - accuracy: 0.1581 - loss: 2.4527  
Epoch 1: val_loss improved from inf to 2.40718, saving model to /content/drive/Shareddrives/OmniClick Team/notebooks/weights/mobilenetv2_unity_best.keras

Epoch 1: saving model to /content/drive/Shareddrives/OmniClick Team/notebooks/weights/mobilenetv2_unity_last.keras
8/8 ━━━━━━━━━━━━━━━━━━━━ 9015s 1215s/step - accuracy: 0.1619 - loss: 2.4302 - val_accuracy: 0.2190 - val_loss: 2.4072
Epoch 2/100
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 4s/step - accuracy: 0.3042 - loss: 1.8370
Epoch 2: val_loss improved from 2.40718 to 2.08601, saving model to /content/drive/Shareddrives/OmniClick Team/notebooks/weights/mobilenetv2_unity_best.keras

Epoch 2: saving model to /content/drive/Shareddrives/OmniClick Team/notebooks/weights/mobilenetv2_unity_last.keras
8/8 ━━━━━━━━━━━━━━━━━━━━ 58s 8s/step - accuracy: 0.3055 - loss: 1.8337 - val_accuracy: 0.2691 - val_loss: 2.0860
Epoch 3/100
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 4s/step - accurac

In [ ]:
!tensorflowjs_converter --input_format=tf_saved_model TF_SAVE_MODEL_DIR TFJS_MODEL_DIR

2025-05-25 13:11:47.426210: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1748178707.447858   54494 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1748178707.454436   54494 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
🌲 Try https://ydf.readthedocs.io, the successor of TensorFlow Decision Forests with more features and faster training!
Traceback (most recent call last):
  File "/usr/local/bin/tensorflowjs_converter", line 8, in <module>
    sys.exit(pip_main())
             ^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/tensorflowjs/converters/converter.py", line 959, in pip_main
    main([' '.join(sys.argv[1:])])
  File "/usr/local/lib

In [ ]:
# Evaluate
test_loss, test_acc = model.evaluate(unity_test_gen)
print(f"Test Accuracy: {test_acc:.4f}")

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


KeyboardInterrupt: 

In [ ]:
best_model = tf.keras.models.load_model(BEST_MODEL_PATH)
last_model = tf.keras.models.load_model(LAST_MODEL_PATH)

In [ ]:
model = best_model

test_gen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255).flow_from_directory(
    os.path.join(UNITY_SUBSET_DIR, 'test'),
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False
)

# Predict
y_pred_probs = model.predict(unity_test_gen)
y_pred = np.argmax(y_pred_probs, axis=1)
y_true = unity_test_gen.classes
class_names = list(unity_test_gen.class_indices.keys())

In [ ]:
# 1. Plot training curves
def plot_training_curves(history):
    plt.figure(figsize=(10, 4))
    # Accuracy
    plt.subplot(1, 2, 1)
    plt.plot(history.history['accuracy'], label='Train Acc')
    plt.plot(history.history['val_accuracy'], label='Val Acc')
    plt.title('Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()

    # Loss
    plt.subplot(1, 2, 2)
    plt.plot(history.history['loss'], label='Train Loss')
    plt.plot(history.history['val_loss'], label='Val Loss')
    plt.title('Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()

    plt.tight_layout()
    plt.show()


plot_training_curves(history)

In [ ]:
# 2. Classification report
report = classification_report(y_true, y_pred, target_names=unity_class_names, output_dict=True)
report_df = pd.DataFrame(report).transpose()
report_df

In [ ]:
# 3. Confusion matrix
cm = confusion_matrix(y_true, y_pred)
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', xticklabels=class_names, yticklabels=class_names, cmap='Blues')
plt.title('Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('True')
plt.show()

In [ ]:
# 4. Inference examples
def plot_inference_examples(generator, y_pred, class_names, num_examples=8):
    fig, axes = plt.subplots(2, num_examples // 2, figsize=(16, 5))
    axes = axes.flatten()
    for i in range(num_examples):
        img, label = generator[i]
        pred_label = class_names[np.argmax(model.predict(np.expand_dims(img[0], axis=0)))]
        true_label = class_names[np.argmax(label[0])]
        axes[i].imshow(img[0])
        axes[i].set_title(f"True: {true_label}\nPred: {pred_label}")
        axes[i].axis('off')
    plt.tight_layout()
    plt.show()

# Display
plot_inference_examples(unity_test_gen, y_pred, class_names)